<H2> Preparando o ambiente</H2>

In [3]:
## Descomente as linhas [!pip install ...] caso precise instalar algumas das bibliotecas utilizadas

## Instalar pacotes do requests
!pip install -q requests

## Instalar pacotes do requests
!pip install -q pandas

## Instalar Watermark
!pip install -q watermark

## Instalar o psycopg2-binary -- para suprir dependência da lib sqlalchemy
!pip install psycopg2-binary -q

In [1]:
# Bibliotecas utilizadas

import sqlalchemy as db     # Acessar e gerenciar BD
import requests             # Obter dados API - WEB
import pandas as pd         # Manipular dados

# *Caso não seja possível importar alguma das bibliotecas acima,
#  descomente a linha correspondente na célula acima

In [4]:
#versões dos pacotes usados neste notebook
%reload_ext watermark
%watermark --iversions

pandas    : 2.2.2
sqlalchemy: 2.0.30
requests  : 2.32.2



#####   VERSÃO DAS LIB'S UTILIZADAS     ####
- pandas    : 2.2.2
- sqlalchemy: 2.0.30
- requests  : 2.32.2

---
<H2> Configurando as conexão

In [8]:
# Definindo variáveis de conexão

## Fonte de dados
api_data = 'https://jsonplaceholder.typicode.com/todos/'

## Conexão BD
db_url = db.URL.create(
    "postgresql",
    username="postgres",
    password="Password123",
    host="localhost",
    port=5432,
    database="datalake",
    )

# Criando conexão com BD Postgres
engine = db.create_engine(db_url)

<H2> Baixando dados da API

In [9]:
# Obtendo os dados da API
response = requests.get(api_data)
data = response.json()
df = pd.DataFrame(data)